In [1]:
# 각 층을 클래스로 설정
class MultiPerceptron:
    def __init__(self, last_layer=None, node=0, activation_param="relu", name=None):
        """_summary_
        Args:
            last_layer: 이전 층, 이전층의 데이터 값과 가중치를 통해 입력값을 계산하기 위해 가져옴
            node: 사용자 설정 값, 원하는 노드 수 설정
            activation_param: 활성화 함수 string 변수
            name: 각 층의 이름 설정
        """
        self.tolerance = 1e-15    # 허용 오차
        
        # 각 활성화 함수 string변수와 함수 매핑
        self.activate_func = {
            "relu": self.relu,
            "sigmoid": self.sigmoid,
            "step": self.step,
            "tanh": self.tanh,
            "identity": self.identity,
            "softmax": self.softmax
        }
        
        self.last_layer = last_layer # 이전 층
        # activation_param을 통해 함수 매핑하여 넣어준다.
        self.activate = self.activate_func[activation_param]  
        self.hidden_node = node # 은닉층의 노드의 갯수
        self.in_data = None     # 이전 층의 데이터와 가중치의 행렬 계산한 값이 들어온다.
        self.w = None           # 현재 층의 가중치
        self.out_data = None    # 입력으로 들어온 데이터를 활성화 함수를 거치게 한 값 다음 층이나 결과가 된다.
        
        self.name=name # Debug용, 각 은닉층의 이름 지정
    
    
    def neuron(self):
        """_summary_
            퍼셉트론의 계산을 진행하는 주요 함수
            이전층의 출력값과 가중치 값의 행렬 계산 => 입력 => 활성화 함수 => 출력 => (다음 층)
        """
        # 이전 레이어와 가중치의 곱으로 퍼셉트론 계산
        tmp = self.layer_calc()
        
        # 활성화 전 곱으로 들어온 값, 입력 데이터에 저장
        self.in_data = tmp
        
        # 입력 데이터 활성화해서 넘겨줌
        tmp = self.activate_loop(tmp)
        
        # 활성화를 거친 값을 출력 값으로 지정
        self.out_data = tmp
    
    
    def layer_calc(self):
        """_summary_
        이전 층의 출력값과 가중치 값의 행렬 계산
        Returns: 행렬 계산 값
        """
        # 계산 후
        x_b = np.c_[self.last_layer.out_data, np.ones(len(self.last_layer.out_data))]
        tmp = x_b.dot(self.last_layer.w)
        
        return tmp
    
    def activate_loop(self, tmp):
        """_summary_
        각 데이터 셋에 대해서 적용 시키기위한 함수
        Args: tmp: 행렬 계산된 값, 
        Returns:  활성화 함수를 거친 값
        """
        for i, data in enumerate(tmp):
            tmp[i] = self.activate(data)
        return tmp
                
    # 아래는 활성화 함수
    # 시그모이드 함수
    def sigmoid(self, x): 
        x = 1 / ( 1 + np.exp(-x))
        return x

    # 계단 함수
    def step(self, x):
        return x > 0

    # 하이퍼볼릭 탄젠트 함수
    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))\
                / (np.exp(x) + np.exp(-x))

    # 항등 함수
    def identity(self, x):
        return x
    
    # Softmax 함수
    def softmax(self, x):
        e = np.exp(x - np.max(x))
        s = np.sum(e)
        return e / s
    
    # ReLU함수
    def relu(self, x):
        return list(map(lambda d: d if d>0 else 0, x))
    
# 모델
class customModel:
    """_summary_
        모든 퍼셉트론 층을 담아두는 함수
    """
    def __init__(self,x=[], y=[], mean=0, mu=2):
        """_summary
        Args:
            x: 입력 데이터, 입력데이터는 항상 numpy array로 준다.
            y: 레이블 데이터
            mean mu, 랜덤값을 위한 평균과 분산
            hidden_layer: 은닉층의 수
        """
        # 모델 전체 layer와 가중치 담아두는 리스트
        self.layer= []
        self.weight = []
        
        # 랜덤하게 가중치 생성을 위한 평균과 분산
        self.mean = mean
        self.mu = mu
        
        self.label=y        # 라벨 값
        self.in_data = x    # 처음 데이터 셋
        self.out_data = []  # 출력을 위한 값
        self.y_pred = []    # 출력을 통해 예측한 값
        
        self.input_feature_cnt = 0 # 입력 데이터의 특성 수
        self.output_class_cnt = 0  # 출력 결과의 클래스 수
        self.hidden_layer_cnt = 0  # 은닉층의 수
        
        self.check_feature_class_cnt() # 입력 데이터의 특성 수와 출력 결과 클래스 수 파악
      
    # 퍼셉트론을 통해 도출한 결과 반환하는 함수          
    def return_output(self):
        return self.out_data
    
    # softmax함수를 나온 값을 통해 결과 값 도출
    def softmax_predict(self):
        for d in self.out_data:
            # 입력 데이터마다 각각의 최대 값의 인덱스를 통해 결과 값 도출
            self.y_pred.append(np.argmax(d)+1)
    
        return self.y_pred
    
    # 설계한 은닉층과 출력층을 실질적으로 데이터를 넣어 실행시키는 함수 
    def fit(self):
        """
            입력 데이터 : row, col
        """
        
        # 모델로 들어온 데이터 입력 데이터에 넣는다.
        # layer0 = 입력층, 입력층은 아무 것도 설정하지 않으므로 입력 데이터 -> 출력 데이터
        self.layer[0].in_data = self.in_data
        self.layer[0].out_data = self.in_data
        
        # 설계한 은닉층에서 출력층까지 돌린다.
        # MultiPerceptron 클래스에서 내부적으로 이전 layer를 가지고 있으므로 돌려주기만 하면 된다.
        for i in range(1,self.hidden_layer_cnt+2):
            self.layer[i].neuron()
        
        # 마지막 출력층의 활성화 함수까지 거친 데이터를 최종 출력 데이터에 넣는다. 
        self.out_data = self.layer[-1].out_data
        
    
    # 모델 클래스 생성시 입력데이터를 통해 입력 특성 수 지정하고 
    # 라벨데이터를 통해 출력 클래스 개수 정하는 함수
    def check_feature_class_cnt(self):
        # numpy인 경우에만 사용
        if type(self.in_data) is np.ndarray:
            self.input_feature_cnt = len(self.in_data[0])
            
        if type(self.label) is np.ndarray:
            self.output_class_cnt = len(self.label[0])
    
    # 가중치 세팅
    def set_weight(self, auto=True):
        """_summary_
        Args:
            auto: True=자동으로 가중치값을 설정해 준다. False=실습2번
        """
        # 실습 2번을 위한 자동 설정 값
        if not auto:
            index = 0
            # Layer 1
            self.weight.append(np.array([
                [0.1,0.2,0.3],
                [0.1,0.3,0.5],
                [0.2,0.4,0.6]
            ]))
            self.layer[index].w = self.weight[index]

            index += 1
            # Layer 2
            self.weight.append(np.array([
                [0.1,0.2],
                [0.1,0.4],
                [0.2,0.5],
                [0.3,0.6]
            ]))
            self.layer[index].w = self.weight[index]
            return

        # AUTO
        # Input
        # 입력층 - 은닉1층 
        # matrix = 입력 특성수 x 은닉1층의 노드 수
        w =np.random.normal(self.mean,self.mu, size=(self.input_feature_cnt + 1,self.layer[1].hidden_node))
        self.weight.append(w) # 설정한 가중치 값 model에 list로 저장
        self.layer[0].w = w   # 각 layer(MultiPerceptron 클래스)에 자신의 가중치 저장
        
        # Hidden - Output
        # matrix = 은닉층의 노드 x 출력층 클래스 수
        # 은닉층만 적용
        for i in range(1, self.hidden_layer_cnt+1):
            w =np.random.normal(self.mean,self.mu, size=(self.layer[i].hidden_node+1,self.layer[i+1].hidden_node))
            self.layer[i].w = w # 자신의 클래스에 값 저장
            self.weight.append(w) # 설정한 가중치 값 model에 list로 저장 
        
    # 실습#2 Case 1
    def set_layer_case1(self):
        # input = 이전 레이어
        # node = 원하는 입력 층 수
        # name = 층의 이름 설정
        # activation_param = 활성화 함수, 자신한테 들어온 데이터를 활성화 함수를 통해 비선형적이게 만들어 다음 층으로 넘긴다. 
        
        # 입력 층 설정
        # 입력층의 노드 = 입력 데이터의 특성 수
        input = MultiPerceptron(node=self.input_feature_cnt, name="Input_Layer")
        self.layer.append(input)
        
        # 은닉층1: {활성화함수:시그모이드, node=3}으로 지정
        layer1 = MultiPerceptron(input, node=3, activation_param="sigmoid", name="Layer1")
        self.layer.append(layer1)
        
        # 출력층: {활성화함수:identity(항등), node=출력 클래스 갯수}으로 지정
        output = MultiPerceptron(layer1, node=self.output_class_cnt, activation_param="identity", name="Output_Layer")
        self.layer.append(output)
        
        # 모든 레이어 - 입력 - 출력
        self.hidden_layer_cnt = len(self.layer) - 2
        
        # 모델 설계후 각 지정한 노드를 통해 가중치 설정
        self.set_weight(auto=False)
        
     # 실습#2 Case 2
    def set_layer_case2(self):
         # 입력 층 설정
        # 입력층의 노드 = 입력 데이터의 특성 수
        input_data = MultiPerceptron(node=self.input_feature_cnt, name="Input_Layer")
        self.layer.append(input_data)
        
        # 은닉층1: {활성화함수:시그모이드, node=3}으로 지정
        layer1 = MultiPerceptron(input_data, node=3, activation_param="sigmoid", name="Layer1")
        self.layer.append(layer1)
        
        # 출력층: {활성화함수:softmax, node=출력 클래스 갯수}으로 지정
        output = MultiPerceptron(layer1, node=self.output_class_cnt, activation_param="softmax", name="Output_Layer")
        self.layer.append(output)
        
        # 모든 레이어 - 입력 - 출력
        self.hidden_layer_cnt = len(self.layer) - 2
        
        # 모델 설계후 각 지정한 노드를 통해 가중치 설정
        self.set_weight(auto=False)
        
    # 실습#2 Case 3  
    def set_layer_case3(self):
        # 입력 층 설정
        # 입력층의 노드 = 입력 데이터의 특성 수
        input_data = MultiPerceptron(node=self.input_feature_cnt, name="Input_Layer")
        self.layer.append(input_data)
        
        # 은닉층1: {활성화함수:relu, node=3}으로 지정
        layer1 = MultiPerceptron(input_data, node=3, activation_param="relu", name="Layer1")
        self.layer.append(layer1)
        
        # 출력층: {활성화함수:identity, node=출력 클래스 갯수}으로 지정
        output = MultiPerceptron(layer1, node=self.output_class_cnt, activation_param="identity", name="Output_Layer")
        self.layer.append(output)
        
        # 모든 레이어 - 입력 - 출력
        self.hidden_layer_cnt = len(self.layer) - 2
        # 모델 설계후 각 지정한 노드를 통해 가중치 설정
        self.set_weight(auto=False)
        
    # 실습#2 Case 4
    def set_layer_case4(self):
        # 입력 층 설정
        # 입력층의 노드 = 입력 데이터의 특성 수
        input_data = MultiPerceptron(node=self.input_feature_cnt, name="Input_Layer")
        self.layer.append(input_data)
        
        # 은닉층1: {활성화함수:relu, node=3}으로 지정
        layer1= MultiPerceptron(input_data, node=3, activation_param="relu", name="Layer1")
        self.layer.append(layer1)
        
        # 출력층: {활성화함수:softmax, node=출력 클래스 갯수}으로 지정
        output = MultiPerceptron(layer1, node=self.output_class_cnt, activation_param="softmax", name="Output_Layer")
        self.layer.append(output)
        
        # 모든 레이어 - 입력 - 출력
        self.hidden_layer_cnt = len(self.layer) - 2
        # 모델 설계후 각 지정한 노드를 통해 가중치 설정
        self.set_weight(auto=False)
      
    # 실습 #3 
    def set_layer(self):
        # 입력 층 설정
        # 입력층의 노드 = 입력 데이터의 특성 수
        input_data = MultiPerceptron(node=self.input_feature_cnt, name="Input_Layer")
        self.layer.append(input_data)
        
        # 은닉층1: {활성화함수:tanh, node=7}으로 지정
        layer1= MultiPerceptron(input_data, node=10, activation_param="tanh", name="Layer1")
        self.layer.append(layer1)
        
        # 출력층: {활성화함수:softmax, node=출력 클래스 갯수}으로 지정
        output = MultiPerceptron(layer1, node=self.output_class_cnt, activation_param="softmax", name="Output_Layer")
        self.layer.append(output)
        
        # 모든 레이어 - 입력 - 출력
        self.hidden_layer_cnt = len(self.layer) - 2
        
        # 모델 설계후 각 지정한 노드를 통해 가중치 설정
        self.set_weight()